# Simple C code mapping (extension) with ctypes 

## Introduction

[Python](https://python.org)은 쉽고 간단한 인터프리터 언어로 현재 다양한 방면에서 널리 사용되고 있다.  
하지만 잘 알려져있는 단점으로 pure python 코드가 바이트코드로 해석해 동작하기 때문에  
특히, 반복문의 속도저하가 상당히 크게 작용한다.  

다행히, Python에서는 이러한 문제를 해결하기 위해 많은 라이브러리들이 존재하고 그러한 라이브러리 내부에는  
C언어로 작성된 코드들을 사용하는 형태로 속도 최적화를 이뤄낸다.  

다만, 일반적이 유저들이 사용하기에 C 함수의 Python 매핑은 조금 까다로운 면이 있다.  
파이썬에서는 [ctypes](https://docs.python.org/ko/3/library/ctypes.html)라는 내장 라이브러리를 제공하여 C 확장을 할 수 있도록 만들었다.  
계산과학에서는 [Numpy](https://numpy.org)를 이용하여 많은 계산을 진행하고 있고,   
numpy 라이브러리에서도 ctypes를 이용한 C 확장을 제공하기 때문에 이를 이용하여 *데이터 버스*의 형식으로 사용하는 것이 편리하다.  

>하지만 이러한 과정도 여러 복잡한 과정을 거치기 때문에 가장 간단히 C 소스코드를 가지고  
바로 Python 매핑을 할수 있도록 간단한 예제들에 대하여 C 함수의 Python 매핑 라이브러리를 작성하였다.

**Keyword** : C extension, optimization, python slow down  
**Reference** : [cffi](https://cffi.readthedocs.io/en/latest/), [numba](https://numba.readthedocs.io/en/stable/index.html), [ctypes](https://docs.python.org/ko/3/library/ctypes.html), [swig](http://www.swig.org/), [cppyy](https://cppyy.readthedocs.io/en/latest/)

In [1]:
from uos_statphys import C_ext

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
import uos_statphys

## 0. Installation





### 0.1 Prerequisite

 clang (by llvm)


### Installation

In [ ]:
!pip install uos_statphys

## 1. Overview

기본적으로 C extenstion의 build과정은 다음과 같다.

>-----   C   ------  
1. C souce code  
2. Compile as shared library  
----- python ------  
3. load library with ctypes in python  
4. specifying arguments type and result type of each function  
5. prepare arguments to use C function  

이 라이브러리에서는 위의 과정을 한가지로 축약한다.  
>-----   C   ------  
1. C souce code with simple decoration  
2. using it in python  

## 2. Symbol for python mapping

Python에서 C코드를 사용하기 위해서는 C 컴파일러의 공유 라이브러리와 같은 방법으로 함수를 불러온다.
공유 라이브러리에 존재하는 함수명을 불러오기 때문에 C++ 코드의 경우에는 C에서 사용가능한 함수로 작성해 주어야 한다.  

또, Python에서는 공유 라이브러리 내부의 함수의 형태를 인식할 수 없기 때문에  
각 함수의 *return type*과 *argument type*을 명시해 주어야 python object를 해당 C type으로 전달해 줄수 있다.  

이 문제를 해결하기 위해서 라이브러리에서는 마치 C 컴파일러의 전처리기와 같이 C 코드를 직접 읽어들이고  
분석하여 위의 작업들을 자동을 처리한다.  

이를 위한 3가지 문법을 아래에서 소개한다.

- extern_python

> 파이썬으로 매핑할 함수들의 헤더가 들어갈 자리를 표시한다. 이는 기존 헤더와 충돌할 수 있기 때문에  
미리 선언해둔 헤더가 끝나는 줄에 사용한다.  

Ex) 

```!C
...
<some pre-declared functions>
...
<the last line of header>

//@extern_python
```


- python_def  

> 파이썬으로 매핑할 함수 표시.  
함수가 실제 정의되어 있는 곳의 바로 윗줄에 C-style 주석으로 표기한다. 다음에 정의된 함수를 Python으로 매핑한다고 표시하는 구문이다.  

Ex) 
```
//@python_def
int some_function(int argument1, int* argument2m ...){
    <code>
    return ret_val;
}
```

- //!  

> 파이썬에서 보이게될 주석 표시. python_def 구문의 밑에 작성한다.(optional)  
파이썬에서는 표준규약으로 함수의 내용을 정리해서 주석으로 작성하는 것을 권장하고 있고  
이를 인터프리터가 읽어들여 저장하고 있다가 실제 함수를 사용할 때 보여주거나 언제든지 확인할수 있다.  
파이썬에 전달할 docstring을 작성하면 C 함수를 매핑할 때 여기에 작성된 docstring도 같이 전달한다.

Ex) 
```
//@python_def
//!Short summary of this function.
//!argument1 is ...
//!argument2 is ...
//!return is ... 
int some_function(int argument1, int* argument2m ...){
    <code>
    return ret_val;
}
```

- //$

> 소스코드의 컴파일 명령어 주석. 이를 이용해 해당 코드를 컴파일 한다.(optional)
각 소스코드는 해당 소스코드를 컴파일 하기 위한 컴파일 명령어가 필요한 경우가 많고 이를 저장해두는 경우가 있다.  
이 라이브러리에서는 위의 형태로 시작하는 single-line comment가 등장하면 해당 줄에서부터 컴파일러와 컴파일 옵션을 서치하여  
컴파일을 시도하게 된다. 일반적인 경우와는 달리 파이썬에서 사용가능한 파일은 공유 라이브러리(`.dll`, `.so`, `.dylib`)이므로  
공유 라이브러리로 만드는 옵션을 추가하여 공유라이브러리로 컴파일하여 불러오게 된다.  
이 라이브러리와는 별개로 이러한 주석을 달아놓는 것은 후에 파일관리를 할때 매우 편리할수 있다.

Ex)
```
//$g++ -o test.out testcode.cpp -std=c++14 
#include<stdio.h>
...
```


### 2.1 Eaxmple of C code

예제 코드는 아래와 같다.  
C 언어로 총 4가지의 함수를 정의하였다.
- add
- array_add
- dot_product
- fibonacci_array

위에서 정의한 문법들을 실제로 어떻게 사용되는 지에 대한 예제도 포함 되어있다.

In [4]:
C_code = """
//$g++ -std=c++14 -o test.out testcode.cpp -Wall
#include <stdio.h>
#include <iostream>
#include <vector>


int add(int, int);
void array_add(int*, int*, int*, int);
void fibonacci_array(int*, int);

//@extern_python

using namespace std;

// some c++ code

//@python_def
int add(int a, int b){
    return a + b;
}

//@python_def
void array_add(int* c, int* a, int* b, int n){
    for (size_t i = 0; i < n; i++)
    {
        c[i] = a[i] +b[i] + 1;
    }
}

//@python_def
int dot_product(int* a, int* b, int n){
    int sum = 0;
    for (size_t i = 0; i < n; i++)
    {
        sum = sum +  a[i] * b[i];
    }
    return sum;
}



//@python_def
//!Get n-th fibonacci series.
//! Parameters
//! -----------
//! result : nd.array
//!    the array will be filled with fibonacci series by this function.
//! n : int
//!    the length of array
void fibonacci_array(
    int* result, 
    int n
    ){
    for (size_t i = 0; i < n; i++)
    {
        if (i>1){
            result[i] = result[i-1]+result[i-2];
        }
        else{
            result[i] = 1;
        }
    }
}

int main(void){
    cout<<"Test File!"<<endl;
    return 0;
}
"""


In [5]:
with open("testcode.cpp", 'w') as f:
    f.write(C_code)

## 2. C type mapping

Python은 기본적으로 object로 구성되어있는 언어이기 때문에 둘 사이에 변환은 필수적인 사안이다.  
(단적으로 이야기하자면,  Python의 `int`와 C에서의 `int`는 같지 않다.)  
따라서, 앞서 이야기했듯이, Python에서 C 함수를 호출할때 전달해주는 인자들을 적절히 변환을 거쳐야 한다.  
함수의 인자의 선언을 보고 아래의 표에 따라 Python의 변수를 해당 c type으로 변환한다.  
(이밖의 C type은 현재 지원하지 않습니다. 지원이 필요할 경우 메일로 회신 부탁드립니다. (Sturcture 포함.))

추가로, 여기서는 모든 pointer를 numpy array로 매핑한다.  

In [6]:
import ctypes
import numpy as np
# c and python interface.
C_ext.c_to_py 

{'int': ctypes.c_long,
 'long': ctypes.c_long,
 'float': ctypes.c_float,
 'float&': ctypes.c_float,
 'double': ctypes.c_double,
 'bool': ctypes.c_bool,
 'float*': numpy.ctypeslib.ndpointer_<f4,
 'char*': numpy.ctypeslib.ndpointer_|i1,
 'int*': numpy.ctypeslib.ndpointer_<i4,
 'long*': numpy.ctypeslib.ndpointer_<i8,
 'double*': numpy.ctypeslib.ndpointer_<f8,
 'int**': numpy.ctypeslib.ndpointer_<f8_2d,
 'void': None}

혹시 이 밖에 사용해야하는 argument types이 있다면 해당 type을 ctypes 모듈 혹은 numpy ctypeslib을 참고하여   
알맞은 타입을 새로 지정해주면 사용가능하다.  
(특정 함수의 특정 인자의 변환에 대한 규칙을 따로 정할 수 있는 문법은 아직 개발되지 않음.)

In [7]:
C_ext.c_to_py['uint'] = ctypes.c_uint

### Caution

주의사항
**Caution** : python에서 C 함수를 호출할때마다 위의 형 변환이 이루어지기 때문에 함수 호출에 ~10 $\mu$s 정도 소요된다.

반복적으로 사용해야하는 함수의 경우 그런 C 함수를 짜서 불러오는 것이 속도면에서 유리하다.

## 3. Usage

지금까지 소개한 문법으로 작성된 C코드를 집어 넣고, compile option을 적어줄수 있다.  
기본 컴파일러는 
- Windows : clang (by llvm)
- linux, OSX : gcc (g++)

로 정의되어 있고, 이를 변경하기 위해서는 `set_compiler(c = None, cpp = None)` 함수를 통해 해당 언의 컴파일러 이름을 써서 변경할 수 있다.

라이브러리는 해당 소스파일을 아래와 같이 컴파일한다.
Ex)
```
target : testcode.cpp
{making} __temp__testcode.cpp
{compiler} -o libtestcode.{outext} __temp__testcode {Compiler options}
```


In [8]:
C_ext.set_compiler(c = 'cl', cpp = 'clang++')

라이브러리에서 해당 코드를 이용해 compile 을 진행한다. 

In [9]:
cdll = C_ext.from_C_source("testcode.cpp", "-O2", debug = True)

C functions found from source : 
	add
	array_add
	dot_product
	fibonacci_array
Compile : clang++ -o __libtestcode.dll __temp__testcode.cpp -shared -O2


compile error가 발생할 수 있다.

In [10]:
cdll

In [11]:
print(cdll.fibonacci_array.__doc__)

Get n-th fibonacci series.
 Parameters
 -----------
 result : nd.array
    the array will be filled with fibonacci series by this function.
 n : int
    the length of array



불러들여진 함수는 C 함수에서의 정의를 그대로 읽어 같은 형태와 이름의 함수를 정의해 제공한다.

In [12]:
cdll.fibonacci_array

<function uos_statphys.C_ext.functions.<locals>.fibonacci_array(result: numpy.ndarray, n: int) -> None>

In [13]:
a = np.empty([10], dtype = np.int32)
cdll.fibonacci_array(a, len(a))
print(a)

[ 1  1  2  3  5  8 13 21 34 55]


## 4. Performance

In [14]:
from numba import njit

In [15]:
@njit
def fibonacci_array(a, n):
    for i in range(n):
        if i>1:
            a[i] = a[i-1] + a[i-2]
        else:
            a[i] = 1

In [16]:
size = 10000

- Numba Just-In-Time compile optimization

In [17]:
%%timeit 
a = np.empty([size], dtype = np.int32)
fibonacci_array(a, len(a))

12.5 µs ± 75 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- Ctypes 

> Numba에 비해 마이크로초 정도 차이나는 것을 확인할 수 있다.

In [18]:
%%timeit 
a = np.empty([size], dtype = np.int32)
cdll.fibonacci_array(a, len(a))

16.9 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- pure python

In [19]:
%%timeit 
a = np.empty([size], dtype = np.int32)
fibonacci_array.py_func(a, len(a))

8.35 ms ± 23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 5. Conclusion
Numba의 최적화보다는 느리지만, C 코드를 그대로 가져와 사용할수 있다는 점에서 충분히 이점이 있다고 생각한다.  
이러한 이점은 디버깅, C 코드의 체크 등등 여러 방법으로 사용할 수 있을 것이라고 기대한다.